In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel'
model_name = "gpt-3.5-turbo"
label_column = 'sentence_class'
text_column = 'tweet'
explanation_column = 'sentence_explanation'
few_shot_num = 8
few_shot_selection = "semantic_similarity"
use_current_explanation = False
use_ground_explanation = False
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=True
temperature=0.1
random_shuffle_examples = True
random_shuffle_examples_seed = 1

In [2]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
import sys

In [3]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [7]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name.split('/')[1]}"

In [8]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_' + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [9]:
test_data = pd.read_csv(data_path)

In [10]:
if use_current_explanation and not use_ground_explanation:
    explanation_column = explanation_column + "_prediction"

In [11]:
config = {
    "task_name": task_name,
    "task_type": "classification",
    "dataset": {
        "label_column": label_column,
        "text_column": text_column,
        "explanation_column": explanation_column,
        "delimiter": ","
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": ("You are an expert at identifying foodborne illness incident information. For the given text, "
                            "your task is to evaluate the text to determine if it describes a potential foodborne illness event. "
                            "Use the following examples as a guide for your predictions and format your responses similarly."
                  ),
        "output_guidelines": ('''Your answer will consist of an explanation, followed by the correct answer ("Yes" or "No").'''
                              '''Please answer with "Yes" if it describes a potential foodborne illness event, otherwise answer with "No".'''
                              '''The last line of the response should always be JSON format with one key: {"label": "the correct answer"}.\n'''),
        "labels": [
            "Yes",
            "No"
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "use_current_explanation": use_current_explanation,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Input: {{{text_column}}}\nOutput: Let's think step by step.\n{{{explanation_column}}}\n{{{label_column}}}",
        "chain_of_thought": True
    }
}

config = NewAutoLabelConfig(config)

In [12]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-03-28 05:54:57 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
if use_current_explanation:
    ds = AutolabelDataset(test_data[[text_column, explanation_column, label_column]], config=config)
else:
    ds = AutolabelDataset(test_data[[text_column, label_column]], config=config)
agent.plan(ds)

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-03-28 05:55:00 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:04 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:08 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Output()

2024-03-28 05:55:12 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:12 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:12 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:12 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:13 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:13 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:13 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:13 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:13 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $0.0936 │
│ Number of Examples       │ 10      │
│ Average cost per example │ $0.0094 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at identifying foodborne illness incident information. Within the TWEET-FID dataset, the task is 
to evaluate a tweet in the TWEET-FID dataset to determine if it describes a potential foodborne illness event. Use 
the following examples as a guide for your predictions and format your responses similarly.

Your answer will consist of an explanation, followed by the correct answer ("Yes" or "No").Please answer with "Yes"
if it describes a potential foodborne illness event, otherwise answer with "No".The last line of the response 
should always be JSON format with one key: {"label": "the correct answer"}.


Some examples with their output answers are provided below:

Input: @USER LOL I know! I've had a stomach bug or food poisoning since yesterday afternoon
Output: Let's think step by step.
The word "food poisoning" in the text is a Keyword entity, which is other relevant keyword or term associated with 
foodborne illnesses, such as "food poisoning".
{"label": "Yes"}
Input: @USER I got food poisoning today wbu
Output: Let's think step by step.
No word in the text is relevant to foodborne illness incident.
{"label": "No"}
Input: @USER Sure its food poisoning? Really bad stomach flu going around the whole country. Feels like food 
poisoning.
Output: Let's think step by step.
No word in the text is relevant to foodborne illness incident.
{"label": "No"}
Input: @USER To be honest, it's been a little rough. Food poisoning. Things are looking up, though.
Output: Let's think step by step.
The word "Food poisoning" in the text is a Keyword entity, which is other relevant keyword or term associated with 
foodborne illnesses, such as "food poisoning".
{"label": "Yes"}
Input: @USER Awww :( I hope it clears up. I just got food poisoning I think ~ throwing up all morning :/
Output: Let's think step by step.
The word "throwing up" in the text is a Symptom entity, which is a specific symptom experienced by the affected 
person as a result of the suspected foodborne illness. The word "food poisoning" in the text is a Keyword entity, 
which is other relevant keyword or term associated with foodborne illnesses, such as "food poisoning".
{"label": "Yes"}
Input: @USER food poisoning is the worst! Hope you get better!
Output: Let's think step by step.
The word "food poisoning" in the text is a Keyword entity, which is other relevant keyword or term associated with 
foodborne illnesses, such as "food poisoning".
{"label": "Yes"}
Input: @USER Hoping you are feeling better tonight! Food poisoning is rough! See you Tues. night! TAKE CARE OF 
YOURSELF!!
Output: Let's think step by step.
The word "Food poisoning" in the text is a Keyword entity, which is other relevant keyword or term associated with 
foodborne illnesses, such as "food poisoning".
{"label": "Yes"}
Input: I've had food poisoning all day and I'm just miserable
Output: Let's think step by step.
The word "food poisoning" in the text is a Keyword entity, which is other relevant keyword or term associated with 
foodborne illnesses, such as "food poisoning".
{"label": "Yes"}

Now I want you to label the following example:
Input: @USER As much fun as I can. Woke up with food poisoning or stomach flu. Been bugging me all day #tmi Almost 
done driving for the day
Output: Let's think step by step.

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [14]:
# now, do the actual labeling
ds = agent.run(ds)

Output()

2024-03-28 05:55:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:15 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-28 05:55:15 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:16 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-28 05:55:16 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:19 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-28 05:55:19 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:20 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-28 05:55:20 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-28 05:55:23 ht

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification_report:
              precision    recall  f1-score   support

          No       0.00      0.00      0.00         2
         Yes       0.80      1.00      0.89         8

    accuracy                           0.80        10
   macro avg       0.40      0.50      0.44        10
weighted avg       0.64      0.80      0.71        10

Actual Cost: 0.0129

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.8      │ 10      │ 1.0             │
└──────────┴─────────┴─────────────────┘

In [15]:
metrics = ds.eval()

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃ classification_report                                 ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.8      │ 10      │ 1.0             │               precision    recall  f1-score   support │
│          │         │                 │                                                       │
│          │         │                 │           No       0.00      0.00      0.00         2 │
│          │         │                 │          Yes       0.80      1.00      0.89         8 │
│          │         │                 │                                                       │
│          │         │                 │     accuracy                           0.80        10 │
│          │         │                 │    macro avg       0.40      0.50      0.44        10 │
│          │         │                 │ weighted avg       0.64      0.80      0.71        10 │
│          │         │                 │                                                       │
└──────────┴─────────┴─────────────────┴───────────────────────────────────────────────────────┘

In [16]:
ds.df.to_csv(output_path, index=False)
ds.df.to_pickle(output_path.replace('.csv', '.pkl'))